In [ ]:
!pip install langdetect

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from wordcloud import WordCloud, STOPWORDS
import os
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import nltk
from langdetect import detect
from wordcloud import WordCloud

In [ ]:
yelp = pd.read_csv('/content/drive/MyDrive/restaurant1/restaurant.csv')
yelp.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,2370338,_WTGv5XnA-qb_XD1D7Z0jg,6PgdGb3HrZdsfl2GiULo8w,RESDUcs7fIiihp38-d6_6g,5,0,1,0,After getting food poisoning at the Palms hote...,2012-12-04 03:10:18
1,2370357,JlNnsvMPLK_1-X2hwzK24w,IS9yw8P2uAPBX6FNLLX4KA,RESDUcs7fIiihp38-d6_6g,4,39,21,29,"""A feast worthy of Gods""\n\nBaccarnal Buffet i...",2014-01-17 00:50:50
2,2370373,hBkoWffORRb6aqKhC_Li2A,uZdFsE_aHbFBChgN6Xa8tw,RESDUcs7fIiihp38-d6_6g,4,1,1,1,The crab legs are better than the ones at Wick...,2015-06-08 18:03:09
3,2370411,rbkxvrgZg5kdCL2a66QYmA,8ZWJNAEWsymXDzKx3B0tTQ,RESDUcs7fIiihp38-d6_6g,1,0,0,0,Not worth it! Too salty food and expensive! Th...,2016-12-19 16:15:29
4,2370500,5tw_pedoHVi9bgeiBNsISg,E0sm4Ve7ifanFYeQMcV8Eg,RESDUcs7fIiihp38-d6_6g,5,0,0,0,I would give this infinite stars if I could. M...,2015-07-28 07:13:17


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10417 entries, 0 to 10416
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   10417 non-null  int64 
 1   review_id    10417 non-null  object
 2   user_id      10417 non-null  object
 3   business_id  10417 non-null  object
 4   stars        10417 non-null  int64 
 5   useful       10417 non-null  int64 
 6   funny        10417 non-null  int64 
 7   cool         10417 non-null  int64 
 8   text         10417 non-null  object
 9   date         10417 non-null  object
dtypes: int64(5), object(5)
memory usage: 814.0+ KB


In [ ]:
yelp['text']=yelp['text'].str.lower()

In [ ]:
yelp['text']=yelp['text'].str.replace('[^\w\s]','')

In [ ]:
yelp['text']=yelp['text'].str.replace('\d+','')

In [ ]:
yelp['text']=yelp['text'].str.replace('\n','').replace('\r','')

In [ ]:
#create a new DataFrame that only contains the 5-star and 1-star reviews
yelp_best_worst=yelp[(yelp.stars==5) | (yelp.stars==1)]

yelp_best_worst.reset_index(drop=True, inplace=True) #reset the indices and instead of creating another data frame, let's just do it inplace

x = yelp_best_worst.text #reviews
y = yelp_best_worst.stars #raitings
#print(x) to look at x
#print(y) to look y
print(x.shape)

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=11)

(5303,)


In [ ]:
yelp_best_worst.head()

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,2370338,_WTGv5XnA-qb_XD1D7Z0jg,6PgdGb3HrZdsfl2GiULo8w,RESDUcs7fIiihp38-d6_6g,5,0,1,0,after getting food poisoning at the palms hote...,2012-12-04 03:10:18
1,2370411,rbkxvrgZg5kdCL2a66QYmA,8ZWJNAEWsymXDzKx3B0tTQ,RESDUcs7fIiihp38-d6_6g,1,0,0,0,not worth it too salty food and expensive this...,2016-12-19 16:15:29
2,2370500,5tw_pedoHVi9bgeiBNsISg,E0sm4Ve7ifanFYeQMcV8Eg,RESDUcs7fIiihp38-d6_6g,5,0,0,0,i would give this infinite stars if i could my...,2015-07-28 07:13:17
3,2370586,gmZJrnsIpYIrhPBsaAVjIw,mCHQQxluggGNYOKqpiR94g,RESDUcs7fIiihp38-d6_6g,5,0,0,0,really enjoyed everything they had to offer it...,2015-04-01 16:18:34
4,2371005,9n4EEhhLlaLDX3QQFVoHMw,or1TN9s4LSRfBF6kmzmb5g,RESDUcs7fIiihp38-d6_6g,5,0,0,0,lots of choices good service line wasnt as lon...,2017-03-27 00:42:19


In [ ]:
x

0       after getting food poisoning at the palms hote...
1       not worth it too salty food and expensive this...
2       i would give this infinite stars if i could my...
3       really enjoyed everything they had to offer it...
4       lots of choices good service line wasnt as lon...
                              ...                        
5298    we went to bacchanal on easter sunday as it wa...
5299    came him from la thanksgiving week we decided ...
5300    every time we visit las vegas we go to this bu...
5301    best buffet ever irma was great served us bett...
5302    the selection is amazing and all the food is e...
Name: text, Length: 5303, dtype: object

Tokenization

In [ ]:
#use countvectorizer to create document term matrices from x_train
vect = CountVectorizer(stop_words='english')
#tokanize the document and count the occurences of token and
#return them as a sparse matrix

x_train_dtm=vect.fit_transform(x_train)
#learn the vocubulary dictionary an create term document matrix

#print(x_train_dtm)
#print(x_train_dtm.shape)
x_test_dtm=vect.transform(x_test)
print(x_test_dtm)
#x_text_dtm.shape

  (0, 102)	1
  (0, 1642)	2
  (0, 1804)	1
  (0, 2729)	1
  (0, 4384)	2
  (0, 4592)	1
  (0, 5132)	1
  (0, 5467)	1
  (0, 6108)	1
  (0, 6357)	1
  (0, 7482)	1
  (0, 7720)	1
  (0, 7930)	1
  (0, 8446)	1
  (0, 8550)	2
  (0, 8609)	1
  (0, 10190)	1
  (0, 10361)	1
  (0, 10977)	1
  (0, 12852)	1
  (0, 13452)	1
  (0, 15239)	1
  (0, 15850)	1
  (1, 366)	1
  (1, 1804)	2
  :	:
  (1324, 366)	1
  (1324, 1376)	1
  (1324, 2155)	1
  (1324, 3472)	1
  (1324, 4836)	1
  (1324, 4912)	1
  (1324, 5104)	1
  (1324, 5467)	1
  (1324, 6737)	1
  (1324, 8550)	1
  (1324, 10554)	1
  (1324, 11444)	1
  (1324, 12422)	1
  (1324, 12497)	2
  (1324, 13202)	1
  (1324, 13340)	1
  (1324, 13823)	1
  (1324, 15696)	1
  (1325, 5467)	1
  (1325, 6108)	1
  (1325, 11003)	1
  (1325, 11444)	1
  (1325, 12505)	1
  (1325, 13113)	1
  (1325, 15952)	1


In [ ]:
print(x_test)

3898    a fat mans paradise skinny people too i guess ...
543     great buffet very well organized the food on t...
3899    i feel as if the food quality tastes better du...
4003    this is my alltime favorite buffet in vegasalt...
4632    wow speechless is the word that describes this...
                              ...                        
4557    tldr expensive but youre in vegassooo impressi...
3329    arrive at this restaurant at their were busy d...
3913    okay you pay a lot for this buffet but you get...
370     amazing food selection and our server carlos m...
1419    food is soso service is not good very pricy  f...
Name: text, Length: 1326, dtype: object


In [ ]:
tf = pd.DataFrame(x_train_dtm.toarray(), columns=vect.get_feature_names_out())
tf.head()

,____,___this,_books,aaaahsooo,aaahhh,aaamazinnng,aaammmaaazzingggg,aahim,aall,aamazing,...,慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢啊啊ーー,服务很好价格也实惠,没礼貌的服务,特に茹でたカニと,真心不怎么样,肯定是请来了全球最地道的大厨吧然而才多一位甩国内的自助条街,这里真心没有爱和很糟糕,连粥和油条都做的地地道道萝卜干都是萧山的味道啊,食物还可以但吃饭要排很久的队而且歧视中国人吃饭还要出示护照不是很好找在凯撒一楼赌场的里面牛排还可以但是螃蟹腿海鲜等食物一般没有想象中的好没有醋等调料吃起来可能会腻饮料需要找服务员单独要总体来说就餐体验和环境都不是很好,高いけど
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_train.head()

2469    got there at  pm on a friday night and the lin...
1290    this is one of the best buffets in las vegas  ...
5256    good service good food  very kind restaurant a...
3054    this buffet was amazing  cool relaxed scene  y...
1478    overhyped and underwhelming food quality was s...
Name: text, dtype: object

In [ ]:
#dont lowercase
vect=CountVectorizer(lowercase=False)
x_train_dtm=vect.fit_transform(x_train)
x_train_dtm.shape

(3977, 16529)

In [ ]:
print(vect.get_feature_names_out()[-50:])

['yyyeeesss' 'zag' 'zealand' 'zeit' 'zero' 'zesty' 'zhongzi' 'zi' 'zig'
 'zigzag' 'zinger' 'zip' 'zipped' 'zitis' 'zone' 'zonecold' 'zong' 'zoo'
 'zorro' 'zu' 'zucchini' 'zugänglich' 'zur' 'àlacarte' 'östlich'
 'únicamente' 'über' 'üblich' 'übliche' 'お料理一品一品の見た目もとても綺麗です' 'ステーキ'
 'デザート達には' 'メチャ美味しいし' '中午没有雪蟹腿菜都很一般港式点心说的过去其他的中式菜只能说尚且入口寿司品质还行'
 '中国游客别去chinese' '凯撒宫的自助餐怎么可以这么棒帝王蟹生蚝虾蟹鲜美无比羊排叉烧披萨提拉米苏布丁每一样都想说umm好吃好吃'
 '吃出了泪花' '品揃えも豊富で' '大大満足' '并没有一些中式buffet好其实两小时的限制也让人不愉快前台不友好'
 '慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢啊啊ーー' '服务很好价格也实惠' '没礼貌的服务'
 '特に茹でたカニと' '真心不怎么样' '肯定是请来了全球最地道的大厨吧然而才多一位甩国内的自助条街' '这里真心没有爱和很糟糕'
 '连粥和油条都做的地地道道萝卜干都是萧山的味道啊'
 '食物还可以但吃饭要排很久的队而且歧视中国人吃饭还要出示护照不是很好找在凯撒一楼赌场的里面牛排还可以但是螃蟹腿海鲜等食物一般没有想象中的好没有醋等调料吃起来可能会腻饮料需要找服务员单独要总体来说就餐体验和环境都不是很好'
 '高いけど']


In [ ]:
#Include 1-grams and 2-grams (an n-gram is N-grams is just all comb..)
#or letters of Length n that you can find in your source text
vect = CountVectorizer(ngram_range=(1,2))
x_train_dtm=vect.fit_transform(x_train)
x_train_dtm.shape

(3977, 161129)

In [ ]:
print(vect.get_feature_names_out()[-50:])

['àlacarte' 'àlacarte restaurants' 'östlich' 'östlich japanisch'
 'únicamente' 'únicamente una' 'über' 'über das' 'üblich' 'üblich hohe'
 'übliche' 'übliche eines' 'お料理一品一品の見た目もとても綺麗です'
 'お料理一品一品の見た目もとても綺麗です 特に茹でたカニと' 'ステーキ' 'ステーキ デザート達には' 'デザート達には'
 'デザート達には 大大満足' 'メチャ美味しいし' 'メチャ美味しいし 品揃えも豊富で'
 '中午没有雪蟹腿菜都很一般港式点心说的过去其他的中式菜只能说尚且入口寿司品质还行'
 '中午没有雪蟹腿菜都很一般港式点心说的过去其他的中式菜只能说尚且入口寿司品质还行 并没有一些中式buffet好其实两小时的限制也让人不愉快前台不友好'
 '中国游客别去chinese' '中国游客别去chinese tourists'
 '凯撒宫的自助餐怎么可以这么棒帝王蟹生蚝虾蟹鲜美无比羊排叉烧披萨提拉米苏布丁每一样都想说umm好吃好吃'
 '凯撒宫的自助餐怎么可以这么棒帝王蟹生蚝虾蟹鲜美无比羊排叉烧披萨提拉米苏布丁每一样都想说umm好吃好吃 x遍' '吃出了泪花'
 '吃出了泪花 肯定是请来了全球最地道的大厨吧然而才多一位甩国内的自助条街' '品揃えも豊富で'
 '品揃えも豊富で お料理一品一品の見た目もとても綺麗です' '大大満足' '并没有一些中式buffet好其实两小时的限制也让人不愉快前台不友好'
 '并没有一些中式buffet好其实两小时的限制也让人不愉快前台不友好 这里真心没有爱和很糟糕'
 '慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢啊啊ーー'
 '慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢啊啊ーー ing李敏哦所以你你咋去搜个xp啊'
 '服务很好价格也实惠' '服务很好价格也实惠 excellent' '没礼貌的服务' '特に茹でたカニと' '特に茹でたカニと ステーキ'
 '真心不怎么样' '真心不怎么样 中午没有雪蟹腿菜都很一般港式点心说的过去其他的中式菜只能说尚且入口寿司品质还行'
 '肯定是请来

In [ ]:
vect = CountVectorizer()
x_train_dtm = vect.fit_transform(x_train)
x_test_dtm = vect.transform(x_test)

#Naive Bayes
nb=MultinomialNB()
nb.fit(x_train_dtm, y_train)
y_pred_class =nb.predict(x_test_dtm)

print(metrics.accuracy_score(y_test,y_pred_class))

0.9381598793363499


In [ ]:
def tokenize_test(vect):
    x_train_dtm=vect.fit_transform(x_train)
    print('Features: ', x_train_dtm.shape[1])
    x_test_dtm=vect.transform(x_test)
    nb=MultinomialNB()
    nb.fit(x_train_dtm, y_train)
    y_pred_class = nb.predict(x_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(y_test,y_pred_class))

In [ ]:
my_test_corpus = ' ı am a gulzade. A free black woman'

In [ ]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  16233
Accuracy:  0.9358974358974359


In [ ]:
print(vect.get_stop_words())

frozenset({'call', 'former', 'six', 'many', 'somehow', 'four', 'up', 'at', 'above', 'hence', 'nobody', 'whenever', 'for', 'yet', 'anything', 'are', 'her', 'becomes', 'once', 'them', 'all', 'two', 'very', 'if', 'which', 'enough', 'forty', 'may', 'this', 'thereby', 'both', 'whom', 'had', 'after', 'beside', 'could', 'besides', 'perhaps', 'others', 'first', 'top', 'hers', 'might', 'indeed', 'although', 'became', 'where', 'anyhow', 'themselves', 'elsewhere', 'were', 'herself', 'none', 'whereafter', 'some', 'nowhere', 'show', 'something', 'hasnt', 'he', 'everything', 'thus', 'put', 'name', 'part', 'noone', 'becoming', 'due', 'detail', 'namely', 'hereafter', 'but', 'next', 'whole', 'there', 'me', 'into', 'his', 'would', 'us', 'any', 'what', 'together', 'amoungst', 'should', 'latter', 'is', 'you', 'nor', 'toward', 'i', 'rather', 'whether', 'off', 'seemed', 'mine', 'front', 'empty', 'meanwhile', 'someone', 'few', 'mill', 'again', 'third', 'him', 'move', 'amount', 'seems', 'on', 'an', 'by', 'eve

In [ ]:
#max features
vect = CountVectorizer(stop_words='english', max_features=100)
tokenize_test(vect)

Features:  100
Accuracy:  0.8521870286576169


In [ ]:
print(vect.get_feature_names_out())

['amazing' 'asian' 'bacchanal' 'bar' 'best' 'better' 'breakfast' 'brunch'
 'buffet' 'buffets' 'caesars' 'came' 'chinese' 'come' 'crab' 'day'
 'definitely' 'delicious' 'dessert' 'desserts' 'did' 'didnt' 'different'
 'dinner' 'dishes' 'dont' 'eat' 'eating' 'experience' 'favorite' 'food'
 'fresh' 'gelato' 'going' 'good' 'got' 'great' 'hour' 'hours' 'huge' 'im'
 'items' 'ive' 'just' 'know' 'las' 'legs' 'like' 'line' 'little' 'long'
 'lot' 'love' 'lunch' 'make' 'meat' 'mexican' 'minutes' 'money' 'nice'
 'options' 'oysters' 'pay' 'people' 'person' 'place' 'plates' 'pm'
 'pretty' 'price' 'prime' 'quality' 'really' 'recommend' 'restaurant'
 'rib' 'right' 'say' 'seafood' 'seated' 'section' 'selection' 'service'
 'shrimp' 'station' 'sure' 'sushi' 'table' 'think' 'time' 'tried' 'try'
 'variety' 'vegas' 'wait' 'want' 'way' 'went' 'worth' 'youre']


In [ ]:
vect = CountVectorizer(ngram_range=(1,2), max_features=100000)
tokenize_test(vect)

Features:  100000
Accuracy:  0.9223227752639518


In [ ]:
vect = CountVectorizer(ngram_range=(1,2), min_df=2)
tokenize_test(vect)

Features:  46589
Accuracy:  0.9502262443438914


TextBlob

In [ ]:
from textblob import TextBlob
b=TextBlob('I have  good spelinhg')
print(b.correct())

I have  good spelling


In [ ]:
yazim = TextBlob('aatım gülzade nazılsın') # türkçe için yok
print(yazim.correct())

atom glade nazılsın


In [ ]:
from textblob import Word
w=Word('falability')
w.spellcheck()

[('fallibility', 0.3333333333333333),
 ('capability', 0.3333333333333333),
 ('affability', 0.3333333333333333)]

In [ ]:
from langdetect import detect
print(detect("War doesn't show who's right, just who's left."))
print(detect("Ein, zwei, drei, vier"))
print(detect("Eu  gosto de mulher"))
print(detect("Nasılsınız"))
print(detect("tmm knk"))

en
de
pt
tr
no


In [ ]:
tr_blob = TextBlob(u"Benim çok işim var")
tr_blob.translate(from_lang="tr", to="de")

TextBlob("Ich habe viel Arbeit")

In [ ]:
en_blob = TextBlob(u"I am a free black man loved by Jesus")
en_blob.translate(from_lang='en', to='fr')

TextBlob("Je suis un homme noir libre aimé de Jésus")

In [ ]:
print(yelp_best_worst.text[0])

after getting food poisoning at the palms hotel i was scared to eat at any more buffets but we figured with the high price tag the positive reviews etc it was worth a chance im really glad we gave it a try btw i uploaded a collage of the things we atethe seafood line is by the far the longest i would not recommend showing up in heels if youre female my feet were killing me between the line at the door the line to get sat and the seafood line totally worth it thoughthe prime rib was the best ive ever hadi loved the sweet potato totsi even have phothe music was great and so was our server alma  he was a older gentleman with a mustache and was very attentive and delightfuli will return next trip in more comfortable shoes after a horrible visit at the palms with non stop issues in a matter of  hours your establishment was truly the high light of our trip thank you for an exquisite dinner


In [ ]:
review=TextBlob(yelp_best_worst.text[0])

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
review.words

WordList(['after', 'getting', 'food', 'poisoning', 'at', 'the', 'palms', 'hotel', 'i', 'was', 'scared', 'to', 'eat', 'at', 'any', 'more', 'buffets', 'but', 'we', 'figured', 'with', 'the', 'high', 'price', 'tag', 'the', 'positive', 'reviews', 'etc', 'it', 'was', 'worth', 'a', 'chance', 'im', 'really', 'glad', 'we', 'gave', 'it', 'a', 'try', 'btw', 'i', 'uploaded', 'a', 'collage', 'of', 'the', 'things', 'we', 'atethe', 'seafood', 'line', 'is', 'by', 'the', 'far', 'the', 'longest', 'i', 'would', 'not', 'recommend', 'showing', 'up', 'in', 'heels', 'if', 'youre', 'female', 'my', 'feet', 'were', 'killing', 'me', 'between', 'the', 'line', 'at', 'the', 'door', 'the', 'line', 'to', 'get', 'sat', 'and', 'the', 'seafood', 'line', 'totally', 'worth', 'it', 'thoughthe', 'prime', 'rib', 'was', 'the', 'best', 'ive', 'ever', 'hadi', 'loved', 'the', 'sweet', 'potato', 'totsi', 'even', 'have', 'phothe', 'music', 'was', 'great', 'and', 'so', 'was', 'our', 'server', 'alma', 'he', 'was', 'a', 'older', 'gen

In [ ]:
review.sentences

[Sentence("after getting food poisoning at the palms hotel i was scared to eat at any more buffets but we figured with the high price tag the positive reviews etc it was worth a chance im really glad we gave it a try btw i uploaded a collage of the things we atethe seafood line is by the far the longest i would not recommend showing up in heels if youre female my feet were killing me between the line at the door the line to get sat and the seafood line totally worth it thoughthe prime rib was the best ive ever hadi loved the sweet potato totsi even have phothe music was great and so was our server alma  he was a older gentleman with a mustache and was very attentive and delightfuli will return next trip in more comfortable shoes after a horrible visit at the palms with non stop issues in a matter of  hours your establishment was truly the high light of our trip thank you for an exquisite dinner")]

In [ ]:
review.lower()

TextBlob("after getting food poisoning at the palms hotel i was scared to eat at any more buffets but we figured with the high price tag the positive reviews etc it was worth a chance im really glad we gave it a try btw i uploaded a collage of the things we atethe seafood line is by the far the longest i would not recommend showing up in heels if youre female my feet were killing me between the line at the door the line to get sat and the seafood line totally worth it thoughthe prime rib was the best ive ever hadi loved the sweet potato totsi even have phothe music was great and so was our server alma  he was a older gentleman with a mustache and was very attentive and delightfuli will return next trip in more comfortable shoes after a horrible visit at the palms with non stop issues in a matter of  hours your establishment was truly the high light of our trip thank you for an exquisite dinner")

Stemming and Lemmatization

In [ ]:
stemmer = SnowballStemmer('english')
print([stemmer.stem(word) for word in review.words])

['after', 'get', 'food', 'poison', 'at', 'the', 'palm', 'hotel', 'i', 'was', 'scare', 'to', 'eat', 'at', 'ani', 'more', 'buffet', 'but', 'we', 'figur', 'with', 'the', 'high', 'price', 'tag', 'the', 'posit', 'review', 'etc', 'it', 'was', 'worth', 'a', 'chanc', 'im', 'realli', 'glad', 'we', 'gave', 'it', 'a', 'tri', 'btw', 'i', 'upload', 'a', 'collag', 'of', 'the', 'thing', 'we', 'ateth', 'seafood', 'line', 'is', 'by', 'the', 'far', 'the', 'longest', 'i', 'would', 'not', 'recommend', 'show', 'up', 'in', 'heel', 'if', 'your', 'femal', 'my', 'feet', 'were', 'kill', 'me', 'between', 'the', 'line', 'at', 'the', 'door', 'the', 'line', 'to', 'get', 'sat', 'and', 'the', 'seafood', 'line', 'total', 'worth', 'it', 'thoughth', 'prime', 'rib', 'was', 'the', 'best', 'ive', 'ever', 'hadi', 'love', 'the', 'sweet', 'potato', 'totsi', 'even', 'have', 'photh', 'music', 'was', 'great', 'and', 'so', 'was', 'our', 'server', 'alma', 'he', 'was', 'a', 'older', 'gentleman', 'with', 'a', 'mustach', 'and', 'was'

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
print([word.lemmatize() for word in review.words])

['after', 'getting', 'food', 'poisoning', 'at', 'the', 'palm', 'hotel', 'i', 'wa', 'scared', 'to', 'eat', 'at', 'any', 'more', 'buffet', 'but', 'we', 'figured', 'with', 'the', 'high', 'price', 'tag', 'the', 'positive', 'review', 'etc', 'it', 'wa', 'worth', 'a', 'chance', 'im', 'really', 'glad', 'we', 'gave', 'it', 'a', 'try', 'btw', 'i', 'uploaded', 'a', 'collage', 'of', 'the', 'thing', 'we', 'atethe', 'seafood', 'line', 'is', 'by', 'the', 'far', 'the', 'longest', 'i', 'would', 'not', 'recommend', 'showing', 'up', 'in', 'heel', 'if', 'youre', 'female', 'my', 'foot', 'were', 'killing', 'me', 'between', 'the', 'line', 'at', 'the', 'door', 'the', 'line', 'to', 'get', 'sat', 'and', 'the', 'seafood', 'line', 'totally', 'worth', 'it', 'thoughthe', 'prime', 'rib', 'wa', 'the', 'best', 'ive', 'ever', 'hadi', 'loved', 'the', 'sweet', 'potato', 'totsi', 'even', 'have', 'phothe', 'music', 'wa', 'great', 'and', 'so', 'wa', 'our', 'server', 'alma', 'he', 'wa', 'a', 'older', 'gentleman', 'with', 'a'

In [ ]:
# assume every word is a verb
print([word.lemmatize(pos='v') for word in review.words])

['after', 'get', 'food', 'poison', 'at', 'the', 'palm', 'hotel', 'i', 'be', 'scar', 'to', 'eat', 'at', 'any', 'more', 'buffet', 'but', 'we', 'figure', 'with', 'the', 'high', 'price', 'tag', 'the', 'positive', 'review', 'etc', 'it', 'be', 'worth', 'a', 'chance', 'im', 'really', 'glad', 'we', 'give', 'it', 'a', 'try', 'btw', 'i', 'upload', 'a', 'collage', 'of', 'the', 'things', 'we', 'atethe', 'seafood', 'line', 'be', 'by', 'the', 'far', 'the', 'longest', 'i', 'would', 'not', 'recommend', 'show', 'up', 'in', 'heel', 'if', 'youre', 'female', 'my', 'feet', 'be', 'kill', 'me', 'between', 'the', 'line', 'at', 'the', 'door', 'the', 'line', 'to', 'get', 'sit', 'and', 'the', 'seafood', 'line', 'totally', 'worth', 'it', 'thoughthe', 'prime', 'rib', 'be', 'the', 'best', 'ive', 'ever', 'hadi', 'love', 'the', 'sweet', 'potato', 'totsi', 'even', 'have', 'phothe', 'music', 'be', 'great', 'and', 'so', 'be', 'our', 'server', 'alma', 'he', 'be', 'a', 'older', 'gentleman', 'with', 'a', 'mustache', 'and',

In [ ]:
def split_into_lemmas(text):
  text=str(text).lower()
  words=TextBlob(text).words
  return [stemmer.stem(word) for word in words]

In [ ]:
#split review text into lemmas rather than into words (default)
vect=CountVectorizer(analyzer=split_into_lemmas)
tokenize_test(vect)

Features:  12889
Accuracy:  0.9343891402714932


In [ ]:
print(vect.get_feature_names_out()[-50:])

['z' 'zag' 'zealand' 'zeit' 'zero' 'zesti' 'zhongzi' 'zi' 'zig' 'zigzag'
 'zinger' 'zip' 'ziti' 'zone' 'zonecold' 'zong' 'zoo' 'zorro' 'zu'
 'zucchini' 'zugänglich' 'zur' 'à' 'àlacart' 'östlich' 'únicament' 'über'
 'üblich' 'お料理一品一品の見た目もとても綺麗です' 'ステーキ' 'デザート達には' 'メチャ美味しいし'
 '中午没有雪蟹腿菜都很一般港式点心说的过去其他的中式菜只能说尚且入口寿司品质还行' '中国游客别去chines'
 '凯撒宫的自助餐怎么可以这么棒帝王蟹生蚝虾蟹鲜美无比羊排叉烧披萨提拉米苏布丁每一样都想说umm好吃好吃' '吃出了泪花' '品揃えも豊富で'
 '大大満足' '并没有一些中式buffet好其实两小时的限制也让人不愉快前台不友好'
 '慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢慢啊啊ーー' '服务很好价格也实惠' '没礼貌的服务'
 '特に茹でたカニと' '皿' '真心不怎么样' '肯定是请来了全球最地道的大厨吧然而才多一位甩国内的自助条街' '这里真心没有爱和很糟糕'
 '连粥和油条都做的地地道道萝卜干都是萧山的味道啊'
 '食物还可以但吃饭要排很久的队而且歧视中国人吃饭还要出示护照不是很好找在凯撒一楼赌场的里面牛排还可以但是螃蟹腿海鲜等食物一般没有想象中的好没有醋等调料吃起来可能会腻饮料需要找服务员单独要总体来说就餐体验和环境都不是很好'
 '高いけど']


Using TF-IDF summarize  a yelpreview

In [ ]:
#create a document - term matrix using TF-IDF
vect=TfidfVectorizer(stop_words='english')
dtm=vect.fit_transform(yelp.text)
features=vect.get_feature_names_out()
dtm.shape

(10417, 32156)

In [ ]:
def summarize():
  review_length=0
  while review_length < 300:
    review_id=np.random.randint(0, len(yelp))
    review_text=str(yelp.text[review_id])
    review_length=len(review_text)
  word_scores={}
  for word in TextBlob(review_text).words:
    word=word.lower()
    if word in features:
      word_scores[word ]=dtm[review_id, features.index(word)]

  print('TOP SCORING WORDS: ')
  top_scores=sorted(word_scores.items(), key=lambda x : x[1], reverse=True)
  for word, score in top_scores:
    print(word)

  print('\n' + review_text)